In [ ]:
!pip install xlrd==1.2.0

     |████████████████████████████████| 103 kB 5.5 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
import datetime
import pandas as pd
import requests
import os
import zipfile
import time
import sys
import csv
import requests

csv.field_size_limit(sys.maxsize)

131072

In [ ]:
from google.colab import drive
# drive.mount._DEBUG = True
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd drive/MyDrive/big_data_project/房屋相關資料集

/content/drive/MyDrive/big_data_project/房屋相關資料集


## 房屋相關資料集

In [ ]:
def real_estate_crawler(year, season):
    if year > 1000:
        year -= 1911

    # download real estate zip file
    res = requests.get("https://plvr.land.moi.gov.tw//DownloadSeason?season="+str(year)+"S"+str(season)+"&type=zip&fileName=lvr_landcsv.zip")

    if len(res.content) < 1000:
      print('no data')
    else:

      # save content to file
      fname = str(year)+str(season)+'.zip'
      # open(fname, 'wb').write(res.content)

      # make additional folder for files to extract
      folder = 'real_estate' + str(year) + str(season)
      if not os.path.isdir(folder):
          os.mkdir(folder)

      # extract files to the folder
      with zipfile.ZipFile(fname, 'r') as zip_ref:
          zip_ref.extractall(folder)

    time.sleep(10)

## 爬取近5年實價登錄

In [ ]:
for year in range(2017, 2023):
    for season in range(1,5):
        print(year, season)
        real_estate_crawler(year, season)

2022 1


## 處理手動增加之非季單位資料

In [ ]:
dirs = [d for d in os.listdir() if d[-12:-4] == 'opendata']
for fname in dirs:
  folder = 'real_estate' + fname[:8]
  if not os.path.isdir(folder):
    os.mkdir(folder)
  with zipfile.ZipFile(fname, 'r') as zip_ref:
    zip_ref.extractall(folder)

# 資料前處理

In [ ]:
# 七都代碼 (加入新竹)
seven_capitals_dict = {'a':'台北市', 'b':'台中市', 'd':'台南市', 'e':'高雄市', 'f':'新北市', 'h':'桃園市', 'o':'新竹市', 'j':'新竹縣'}

# 歷年資料夾
dirs = [d for d in os.listdir() if d[:4] == 'real']

## x_lvr_land_a：房屋買賣交易

In [ ]:
a_dfs = list()
for d in dirs:
  for k,v in seven_capitals_dict.items():
    file_name = str(k).strip() + '_lvr_land_a.csv'
    if file_name not in os.listdir(d):
      continue
    a_df = pd.read_csv(os.path.join(d, file_name), index_col=False, engine='python', error_bad_lines=False)

    # 增加縣市資訊
    a_df['縣市'] = str(v).strip() 

    a_dfs.append(a_df.iloc[1:])
a_df = pd.concat(a_dfs, sort=True)
a_df = a_df.drop_duplicates()

# 格式化日期資訊
date_list = list()
exceptions_list = list()
for date in a_df['交易年月日']:
  format_date = str(int(date[:len(date)-4]) + 1911) + '-' + date[len(date)-4:len(date)-2] + '-' + date[len(date)-2:len(date)]
  # 過濾年份小於2017年且月份或日期為零的資料
  if date[len(date)-4:len(date)-2] == '00' or date[len(date)-2:len(date)] == '00' or int(date[:len(date)-4]) < 106:
    exceptions_list.append(format_date)
  date_list.append(format_date)
a_df['交易年月日'] = date_list

print(f'原始資料: {a_df.shape}')
# 過濾不存在的時間資訊
a_df = a_df[~a_df['交易年月日'].isin(exceptions_list)]
pd.to_datetime(a_df['交易年月日'])
# 過濾超過目前時間的資訊
a_df = a_df[a_df['交易年月日'] <= str(datetime.date.today())]
print(f'過濾小於2017年以及不存在的時間後: {a_df.shape}')
# 篩選資料欄位
a_df = a_df[['交易標的', '主建物面積','主要用途', '交易年月日', '單價元平方公尺','土地移轉總面積平方公尺', '建物型態', '建物現況格局-廳', '建物現況格局-房', '建物現況格局-衛','建物移轉總面積平方公尺', '建築完成年月', '縣市', '總價元', '總樓層數', '車位移轉總面積(平方公尺)', '車位總價元', '車位類別','鄉鎮市區','附屬建物面積', '陽台面積', '電梯']]
print(f'篩選資料欄位後: {a_df.shape}')
# 篩選建物型態
building_type = ['華廈(10層含以下有電梯)', '住宅大樓(11層含以上有電梯)', '公寓(5樓含以下無電梯)', '套房(1房1廳1衛)', '透天厝']
building_list = list()
for t in building_type:
  building_list.append(a_df[a_df['建物型態']==t])
a_df = pd.concat(building_list)
print(f'篩選建物型態後: {a_df.shape}')
# 篩選交易標的 (過濾純建物, 車位以及土地)
trans_object = ['房地(土地+建物)', '房地(土地+建物)+車位']
tran_list = list()
for o in trans_object:
  tran_list.append(a_df[a_df['交易標的']==o])
a_df = pd.concat(tran_list)
print(f'篩選交易標的後: {a_df.shape}')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


原始資料: (1450933, 34)
過濾小於2017年以及不存在的時間後: (1349936, 34)
篩選資料欄位後: (1349936, 22)
篩選建物型態後: (1026447, 22)
篩選交易標的後: (1021059, 22)


# Preprocessing

In [ ]:
# 將電梯欄位為 "NaN" 之資料透過建物型態判斷是否有電梯 (例如: 電梯欄位為 "NaN" 且建物型態為 "華廈(10層含以下有電梯)" 將其視為有電梯進行補值)
# 若建物型態為 "套房(1房1廳1衛)" 及 "透天厝" 則先不進行補值

include_elevator = ['華廈(10層含以下有電梯)', '住宅大樓(11層含以上有電梯)']
not_include_elevator = ['公寓(5樓含以下無電梯)']
exception = ['套房(1房1廳1衛)', '透天厝']
elevator_list = list()
for include in include_elevator: # 包含電梯
  elevator_list.append(a_df[~(a_df['電梯'].isnull()==True) & (a_df['建物型態']==include)]) # 非 "NaN" 資料
  elevator_list.append(a_df[(a_df['電梯'].isnull()==True) & (a_df['建物型態']==include)].assign(電梯='有')) # 將 "NaN" 資料進行補值

for not_include in not_include_elevator: # 不包含電梯
  elevator_list.append(a_df[~(a_df['電梯'].isnull()==True) & (a_df['建物型態']==not_include)]) # 非 "NaN" 資料
  elevator_list.append(a_df[(a_df['電梯'].isnull()==True) & (a_df['建物型態']==not_include)].assign(電梯='無')) # 將 "NaN" 資料進行補值

for e in exception: # 不確定
  elevator_list.append(a_df[~(a_df['電梯'].isnull()==True) & (a_df['建物型態']==e)]) # 非 "NaN" 資料
  elevator_list.append(a_df[(a_df['電梯'].isnull()==True) & (a_df['建物型態']==e)].assign(電梯='不確定')) # 將 "NaN" 資料進行補值

a_df = pd.concat(elevator_list)
a_df = a_df.reset_index(drop=True)
a_df['電梯'].unique()

array(['有', '無', '不確定'], dtype=object)

In [ ]:
# 透過交易標的判斷是否有車位 (例如: "房地(土地+建物)+車位" 將其視為有車位)
# 若交易標的為 "房地(土地+建物)" 則視為無車位

a_df['車位'] = None
include_parking = ['房地(土地+建物)+車位']
not_include_parking = ['房地(土地+建物)']
parking_list = list()

for include in include_parking: # 包含車位
  parking_list.append(a_df[(a_df['車位'].isnull()==True) & (a_df['交易標的']==include)].assign(車位='有')) # 將 "NaN" 資料進行補值

for not_include in not_include_parking: # 不包含車位
  parking_list.append(a_df[(a_df['車位'].isnull()==True) & (a_df['交易標的']==not_include)].assign(車位='無')) # 將 "NaN" 資料進行補值

a_df = pd.concat(parking_list)
a_df = a_df.reset_index(drop=True)
a_df['車位'].unique()

array(['有', '無'], dtype=object)

In [ ]:
# 排除 duplicate 錯誤
for i in a_df:
  a_df[i] = a_df[i].astype(str)

In [ ]:
# 過濾總價元為0的交易
a_df = a_df[a_df['總價元']!='0']
# 過濾單價元平方公尺為0的交易
a_df = a_df[a_df['單價元平方公尺']!='0']

In [ ]:
a_df = a_df.drop_duplicates(keep=False)
a_df = a_df.reset_index(drop=True)
a_df.shape

(1009902, 23)

In [ ]:
a_df['縣市'].value_counts()

新北市    227725
台中市    182846
桃園市    165908
高雄市    161979
台北市     97632
台南市     95892
新竹縣     44674
新竹市     33246
Name: 縣市, dtype: int64

In [ ]:
a_df.to_csv('2017_to_present_七都房屋買賣交易.csv', index=False)

## 人口相關資料集

In [ ]:
cd ../人口相關資料集

/content/drive/MyDrive/big_data_project/人口相關資料集


In [ ]:
population_list = list()
seven_capitals_list = ['臺 北 市', '臺 中 市', '臺 南 市', '高 雄 市', '新 北 市', '桃 園 市', '新 竹 市', '新 竹 縣']
dirs = [d for d in os.listdir() if d[:4] == 'm1s2']
for dir in dirs:
  print(f'Processed files: {dir}')
  if dir[5:8] == '111':
    month = 3 # 111 年僅到 3月份
  else:
    month = 12
  for i in range(month):
    if i+1 < 10:
      sheet = '0' + str(i+1)
    else:
      sheet = str(i+1)
    workbook = pd.read_excel(dir, header=None, sheet_name=sheet)

    seven_capitals_indexing = dict()
    for capital in seven_capitals_list:
      process_captial = capital.replace(' ', '').strip()
      seven_capitals_indexing[capital] = []
      index = workbook[workbook[0]==capital].index.values[0]
      seven_capitals_indexing[capital].append(index-1)
      seven_capitals_indexing[capital].append(index)
      seven_capitals_indexing[capital].append(index+1)
    seven_capitals_indexing

    # 取標題及副標題
    title = workbook[3:4]
    subtitle = workbook[2:3]
    count = 5
    for i in range(title.shape[1]):
      if i < 3:
        title[i] = str(subtitle[i].values[0]).replace('\u3000', '').replace(' ', '').strip() 
      else:
        count += 1
        if i == title.shape[1]-1:
          title[i] = subtitle[i].values
          break
        if count == 6:
          tks = str(subtitle[i].values[0]).replace('\u3000', '').replace(' ', '').strip().split('～')
          title[i] = str(int(tks[0])) + '~' + str(int(tks[1]))
          count = 0
        else:
          title[i] = title[i].astype(int)
    columns_name = title[:].values[0].tolist()

    # 提取七都人口資料
    seven_capitals_population_list = list()
    for k, v in seven_capitals_indexing.items():
      for index in v:
        population = workbook[index:index+1]
        population[0] = k.replace(' ', '').strip()
        seven_capitals_population_list.append(population)
    seven_capitals_population_df = pd.concat(seven_capitals_population_list)
    seven_capitals_population_df.columns = columns_name
    seven_capitals_population_df = seven_capitals_population_df.reset_index(drop=True)
    
    # 加入日期資訊
    format_date = str(int(dir[5:8]) + 1911) + '-' + sheet
    seven_capitals_population_df['日期'] = format_date
    population_list.append(seven_capitals_population_df)
seven_capitals_population_df = pd.concat(population_list)

Processed files: m1s2-11100.xls


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Processed files: m1s2-10600.xls
Processed files: m1s2-10700.xls
Processed files: m1s2-10800.xls
Processed files: m1s2-10900.xls
Processed files: m1s2-11000.xls


In [ ]:
seven_capitals_population_df.to_csv('2017_to_present_七都每月年齡與性別人口數量.csv', index=False)

## COVID-19相關資料集

In [ ]:
cd ../COVID-19相關資料集

/content/drive/MyDrive/big_data_project/COVID-19相關資料集


In [ ]:
covid_19_indexing = {"id":"ID","a01":"iso_code","a02":"洲名","a03":"國家","a04":"日期","a05":"總確診數","a06":"新增確診數","a07":"七天移動平均新增確診數","a08":"總死亡數","a09":"新增死亡數","a10":"七天移動平均新增死亡數","a11":"每百萬人確診數","a12":"每百萬人死亡數","a13":"傳染率","a14":"新增檢驗件數","a15":"總檢驗件數","a16":"每千人檢驗件數","a17":"七天移動平均新增檢驗件數","a18":"陽性率","a19":"每確診案例相對檢驗數量","a20":"疫苗總接種總劑數","a21":"疫苗總接種人數","a22":"疫苗新增接種劑數","a23":"七天移動平均疫苗新增接種劑數","a24":"每百人接種疫苗劑數","a25":"每百人接種疫苗人數","a26":"疫情控管指數","a27":"總人口數","a28":"中位數年紀","a29":"70歲以上人口比例","a30":"平均壽命","a31":"解除隔離數","a32":"解封指數"}
r = requests.get('https://covid-19.nchc.org.tw/api/covid19?CK=covid-19@nchc.org.tw&querydata=4001&limited=TWN')
covid_19 = r.json()

In [ ]:
# 將 NCHC 之 COVID-19 資料集轉換為 Dataframe
covid_19_dict = dict()
for i in range(len(covid_19)):
  for k,v in covid_19_indexing.items():
    if v not in covid_19_dict.keys():
      covid_19_dict[v] = list()
    covid_19_dict[v].append(covid_19[i][k])
covid_19_df = pd.DataFrame()
for k,v in covid_19_dict.items():
  covid_19_df[k] = v

In [ ]:
covid_19_df.to_csv('COVID-19台灣疫情報表.csv', index=False)